In [48]:
import zlib
import numpy as np

In [49]:
def leer_archivo(nombre_archivo):
    """Lee el archivo y devuelve los datos estructurados"""
    with open(nombre_archivo, 'r') as f:
        lineas = f.readlines()
    
    n, m = map(int, lineas[0].split())
    ids = lineas[1].strip().split()
    datos = np.array([list(map(int, linea.strip().split())) for linea in lineas[2:2+n]])
    
    return n, m, ids, datos


In [50]:
def codificar_alt1_sin_cuantizacion(datos):
    """Alternativa 1 pura sin cuantización"""
    encoded = np.zeros_like(datos)
    encoded[:, 0] = datos[:, 0]
    for i in range(1, datos.shape[1]):
        encoded[:, i] = datos[:, i] - datos[:, i-1]
    return encoded

In [51]:
def codificar_hibrido_sin_cuantizacion(datos):
    """Híbrido sin cuantización, usando floats nativos"""
    corr_matrix = np.corrcoef(datos)
    ref_index = np.argmax(np.mean(corr_matrix, axis=1))
    referencia = datos[ref_index]
    
    # Diferencias espaciales y temporales
    dif_espacial = datos - referencia
    dif_espacial[ref_index] = referencia
    deltas = np.diff(dif_espacial, axis=1, prepend=0)
    return deltas


In [52]:
def codificar_code2_corrdatos(datos):
    code1 = np.zeros_like(datos)
    code1[:, 0] = datos[:, 0]
    for i in range(1, datos.shape[1]):
        code1[:, i] = datos[:, i] - datos[:, i-1]
    """ Se usa como valor de correlación los datos originales """
    corr_matrix = np.corrcoef(datos)
    ref_index = np.argmax(np.mean(corr_matrix, axis=1))
    print("Indice de referencia:", ref_index)
    referencia = code1[ref_index]
    
    # Diferencias espaciales y temporales
    dif_espacial = code1 - referencia
    dif_espacial[ref_index] = referencia
    deltas = np.diff(dif_espacial, axis=1, prepend=0)
    return deltas

In [53]:
def codificar_code2_corrcode1(datos):
    code1 = np.zeros_like(datos)
    code1[:, 0] = datos[:, 0]
    for i in range(1, datos.shape[1]):
        code1[:, i] = datos[:, i] - datos[:, i-1]
    """ Se usa como valor de correlación los datos originales """
    corr_matrix = np.corrcoef(code1)
    ref_index = np.argmax(np.mean(corr_matrix, axis=1))
    print("Indice de referencia:", ref_index)
    referencia = code1[ref_index]
    
    # Diferencias espaciales y temporales
    dif_espacial = code1 - referencia
    dif_espacial[ref_index] = referencia
    deltas = np.diff(dif_espacial, axis=1, prepend=0)
    return deltas

In [54]:
def comprimir_justo(datos_codificados):
    """Compresión usando representación binaria nativa de floats (64 bits)"""
    return zlib.compress(datos_codificados.tobytes())

In [58]:
filenameGroups = "../procesa_archivos_2024/outputdata_m30_fulldates/archivos_articulo/m30_2024_ids_estructura_intensidad.txt"
with open(filenameGroups, 'r') as f:
    lineas = f.readlines()
gs = list(map(int, lineas[0].split()))
p = 1
for g in gs:
    idsG = lineas[p].split()
    dataG = np.zeros((g, m))
    posG = 0
    for idG in idsG:
        indice = ids.index(idG)
        dataG[posG] = datos[indice]
        posG += 1
    print("Analizando grupo", p)
    print(g, "IDs:", idsG)
    print("Datos:", dataG)
    
    # Codificar
    alt1 = codificar_alt1_sin_cuantizacion(dataG)
    hibrido = codificar_hibrido_sin_cuantizacion(dataG)
    code2fd = codificar_code2_corrdatos(dataG)
    code2fc = codificar_code2_corrcode1(dataG)
    
    # Comprimir
    c_alt1 = len(comprimir_justo(alt1))
    c_hibrido = len(comprimir_justo(hibrido))
    c_code2fd = len(comprimir_justo(code2fd))
    c_code2fc = len(comprimir_justo(code2fc))
    
    # Resultados
    print("\n================== COMPARACIÓN JUSTA (sin cuantización) ==================")
    print(f"Alternativa 1: {c_alt1} bytes")
    print(f"Híbrido: {c_hibrido} bytes ({((c_alt1 - c_hibrido)/c_alt1)*100:.1f}% mejor)")
    print(f"Code 2 (Corr. Datos): {c_code2fd} bytes ({((c_alt1 - c_code2fd)/c_alt1)*100:.1f}% mejor)")
    print(f"Code 2 (Corr. Code 1): {c_code2fc} bytes ({((c_alt1 - c_code2fc)/c_alt1)*100:.1f}% mejor)")
    
    p += 1
    print("==============================================================================")

Analizando grupo 1
28 IDs: ['6641', '6642', '6644', '6648', '6650', '6654', '10175', '3799', '6666', '3491', '3816', '6672', '6679', '6680', '6683', '6685', '6687', '6689', '6691', '6693', '6694', '6695', '6698', '6699', '6700', '3801', '6706', '7013']
Datos: [[ 190.  156.  912. ...  525.  460.  248.]
 [ 196.  205. 1140. ...  740.  588.  356.]
 [ 136.  124.  776. ...  572.  424.  260.]
 ...
 [  72.   76.  368. ...  152.  205.  148.]
 [  56.   56.  319. ...  140.  152.   80.]
 [  80.  132.  116. ...  108.  108.   73.]]
Indice de referencia: 2
Indice de referencia: 8

================== COMPARACIÓN JUSTA (sin cuantización) ==================
Alternativa 1: 1953166 bytes
Híbrido: 2032326 bytes (-4.1% mejor)
Code 2 (Corr. Datos): 2115550 bytes (-8.3% mejor)
Code 2 (Corr. Code 1): 2141550 bytes (-9.6% mejor)
Analizando grupo 2
23 IDs: ['6643', '6645', '6647', '6649', '6651', '6655', '10265', '3819', '3800', '3488', '3490', '3492', '6670', '6673', '6677', '6681', '7125', '6813', '10180', '68